# Clustering
Here im gonna cluster | Semantic Continents

clean up my clusters | not sure what analogy works for this maybe landscaping?

cluster again to get micro clusters  | semantic islands

send various random represenative text from micro clusters to my good friend GPT via OPENAIapi to get hopefully stable theme tagging | travel brochure

then im done with this and can fit some scikit learn models on yfinance data 

In [1]:
from pathlib import Path
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

# ----------------------------------------
# Resolve paths (NOTEBOOK SAFE)
# ----------------------------------------
# Path of this notebook's directory
FILE_DIR = Path().resolve()          # notebook-safe
ROOT = FILE_DIR.parent               # one level up from /clustering
GOLD_DELTA = ROOT / "pipelines" / "gold" / "gold_delta"

print("Notebook directory:", FILE_DIR)
print("Gold Delta path:", GOLD_DELTA)

# ----------------------------------------
# Spark session (safe settings for laptop)
# ----------------------------------------
builder = (
    SparkSession.builder
        .appName("gold_cluster_stage1")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .config("spark.driver.memory", "8g")
        .config("spark.sql.shuffle.partitions", "16")
        .config("spark.driver.maxResultSize", "2g")       # prevents collect() OOM
        .config("spark.sql.files.maxPartitionBytes", "64m") # laptop-friendly partitions
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()
print("Spark session initialised.")

# ----------------------------------------
# Load gold delta
# ----------------------------------------
df = spark.read.format("delta").load(str(GOLD_DELTA))

print("Loaded delta. Rows =", df.count())
df.printSchema()


Notebook directory: /home/david/School/CapStone/clustering
Gold Delta path: /home/david/School/CapStone/pipelines/gold/gold_delta


25/12/06 13:21:57 WARN Utils: Your hostname, david-ThinkPad-T490 resolves to a loopback address: 127.0.1.1; using 172.16.0.186 instead (on interface wlp0s20f3)
25/12/06 13:21:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/david/School/CapStone/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/david/.ivy2/cache
The jars for the packages stored in: /home/david/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-59014793-96d4-4383-b936-ea23ab049391;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 394ms :: artifacts dl 22ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0  

Spark session initialised.


25/12/06 13:22:13 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
25/12/06 13:22:25 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Loaded delta. Rows = 1949081
root
 |-- date: timestamp_ntz (nullable = true)
 |-- text: string (nullable = true)
 |-- publication: string (nullable = true)
 |-- author: string (nullable = true)
 |-- url: string (nullable = true)
 |-- text_type: string (nullable = true)
 |-- time_precision: string (nullable = true)
 |-- date_trading: timestamp_ntz (nullable = true)
 |-- tz_hint: string (nullable = true)
 |-- dataset: string (nullable = true)
 |-- dataset_source: string (nullable = true)
 |-- source: string (nullable = true)
 |-- source_file: string (nullable = true)
 |-- len_text: integer (nullable = true)
 |-- silver_ingestion_ts: timestamp_ntz (nullable = true)
 |-- emb_0: float (nullable = true)
 |-- emb_1: float (nullable = true)
 |-- emb_2: float (nullable = true)
 |-- emb_3: float (nullable = true)
 |-- emb_4: float (nullable = true)
 |-- emb_5: float (nullable = true)
 |-- emb_6: float (nullable = true)
 |-- emb_7: float (nullable = true)
 |-- emb_8: float (nullable = true)
 |-- 

In [2]:
from pyspark.sql import functions as F

# ----------------------------------------
# Define Stage-1 Feature Columns
# ----------------------------------------
stage1_pca    = [f"pca_{i}" for i in range(50)]
stage1_ner    = ["ner_org", "ner_gpe", "ner_person", "ner_money"]
stage1_lm     = ["lm_pos", "lm_neg", "lm_unc", "lm_lit", "lm_con"]
stage1_basic  = ["len_chars_gold", "num_words", "avg_word_len"]

stage1_cols = stage1_pca + stage1_ner + stage1_lm + stage1_basic

print("Stage 1 feature columns:", len(stage1_cols))

# ----------------------------------------
# Select only needed columns
# ----------------------------------------
df1 = df.select(F.col("url"), *(F.col(c) for c in stage1_cols))

# ----------------------------------------
# Lightweight sanity check
# ----------------------------------------
df1.show(5, truncate=False)
print("Row estimate:", df1.count())   # okay ONCE, but avoid re-running


Stage 1 feature columns: 62


+-------------------------------------------------------------------------------------------+------------+------------+-----------+-----------+-------------+-----------+-------------+------------+-----------+-------------+------------+------------+-----------+------------+------------+------------+------------+------------+-----------+-----------+------------+-----------+------------+------------+-----------+-----------+-------------+------------+-------------+------------+------------+-------------+-------------+------------+------------+------------+------------+------------+------------+------------+-----------+-----------+------------+------------+------------+------------+-------------+------------+-------------+-------------+-------+-------+----------+---------+------+------+------+------+------+--------------+---------+------------------+
|url                                                                                        |pca_0       |pca_1       |pca_2      |pca_3      

Row estimate: 1949081


In [3]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

# ----------------------------------------
# VectorAssembler
# ----------------------------------------
assembler = VectorAssembler(
    inputCols=stage1_cols,
    outputCol="features_raw"
)

df2 = assembler.transform(df1)

print("Assembled vector column created.")


# ----------------------------------------
# Standard Scaler — SAFE mode
# ----------------------------------------
# Fit scaler on a sampled subset (e.g., 200k rows)
# This avoids scanning all 2M rows twice.

SAMPLE_SIZE = 200_000

df_sample = df2.sample(withReplacement=False, fraction=min(1.0, SAMPLE_SIZE / df2.count()))

print("Sample collected for StandardScaler fit:", df_sample.count())

scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withMean=True,
    withStd=True
)

scaler_model = scaler.fit(df_sample)   # <-- massively cheaper
df3 = scaler_model.transform(df2)

# ----------------------------------------
# Lightweight check
# ----------------------------------


Assembled vector column created.


Sample collected for StandardScaler fit: 200926


# Macro Clustering and Cluster Cleaning

In [4]:
from pyspark.ml.clustering import KMeans

k = 10 

kmeans = KMeans(
    featuresCol="features",
    predictionCol="macro_cluster",
    k=k,
    seed=67,
    maxIter=40
)

print("Training KMeans...")
k_model = kmeans.fit(df3)
print("KMeans trained.")

df_macro = k_model.transform(df3)

df_macro.select("url", "macro_cluster").show(10, truncate=False)


Training KMeans...


25/12/06 13:46:32 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


KMeans trained.
+------------------------------------------------------------------------------------------+-------------+
|url                                                                                       |macro_cluster|
+------------------------------------------------------------------------------------------+-------------+
|http://iht-retrospective.blogs.nytimes.com/2016/08/10/1891-bicyclists-claim-right-to-road/|3            |
|http://www.reuters.com/article/australia-cyclone-coal-idUSL3N1HD1LP                       |4            |
|http://www.reuters.com/article/brief-doha-insurance-appoints-jassim-ali-idUSFWN1HC0LO     |0            |
|http://www.reuters.com/article/brief-dustin-group-q2-net-sales-up-119-p-idUSFWN1HC0LY     |0            |
|http://www.reuters.com/article/brief-homestreet-inc-announces-appointme-idUSASB0B8NT      |0            |
|http://www.reuters.com/article/brief-ingenic-semiconductors-shares-to-r-idUSL3N1HD2X6     |0            |
|http://www.reuters.c

### Macro Cleaning

-removing crap and poorly clustered articles
-merging like clusters

**methods**

-cosine similartiy filter to centroid below 90% and its wraps for that we send these articles to the evil MISC_cluster (-1)

-if two centroids are 90%+ similar well lets just make them one cluster

I do this because I want cleaner micro clusters and less noise in my final theme_time features


This is perhaps not the most academic process but im kind eyeballing kmeans N here by getting a sample 10 articles from each clustering run per cluster and seeing if its coherent or jibberish to find an optimal K values tested include 50, 30, 20, 15. But you know student project and all <3


following succesfulish macro clustering, my semantic continents are micro clustered with patience and HDBscan, sample articles and headlines are then shipped to my noble workhorse chatgpt via the openAI API, who returns some thoughtful aicrafted tagging for each cluster. the sample is random each time the proxy i evaluate on is cluster stability basically is ChatGPT giving me similar tags across 5 sample runs :thumbs up emoji



In [6]:
import pyspark.sql.functions as F

# PCA columns
pca_cols = [f"pca_{i}" for i in range(50)]

df_centroids = (
    df_macro
    .groupBy("macro_cluster")
    .agg(*[F.avg(c).alias(c) for c in pca_cols])
)

df_centroids.show(5, truncate=False)

df_c = df_macro.join(
    df_centroids.select(
        "macro_cluster",
        *[F.col(c).alias(f"c_{c}") for c in pca_cols]
    ),
    on="macro_cluster",
    how="inner"
)


+-------------+--------------------+---------------------+---------------------+---------------------+----------------------+---------------------+--------------------+---------------------+---------------------+---------------------+----------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+----------------------+----------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+----------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+----------------------+----------------------+---------------------+----------------------+---------------------+---------------------+---------------------+---------------------+-----------

In [7]:
from pyspark.sql.types import DoubleType
import numpy as np
import pyspark.sql.functions as F

def cos_sim_row(v1, v2):
    a = np.array(v1, dtype=float)
    b = np.array(v2, dtype=float)
    denom = np.linalg.norm(a) * np.linalg.norm(b)
    return float(a.dot(b) / denom) if denom != 0 else 0.0

cos_udf = F.udf(cos_sim_row, DoubleType())

df_c = (
    df_c
    .withColumn("vec", F.array(*[F.col(c) for c in pca_cols]))
    .withColumn("centroid", F.array(*[F.col(f"c_{c}") for c in pca_cols]))
)

df_s = df_c.withColumn("cosine_sim", cos_udf("vec", "centroid"))


In [9]:
df_thresh = (
    df_s
    .groupBy("macro_cluster")
    .agg(F.expr("percentile_approx(cosine_sim, 0.20)").alias("sim_cutoff"))
)
df_s2 = df_s.join(df_thresh, on="macro_cluster", how="inner")

df_clean = df_s2.withColumn(
    "macro_clean",
    F.when(F.col("cosine_sim") < F.col("sim_cutoff"), F.lit(-1))
     .otherwise(F.col("macro_cluster"))
)

#merging like centroids
centroids_pd = df_centroids.toPandas()


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

vecs = centroids_pd[pca_cols].values
ids = centroids_pd["macro_cluster"].values
sim = cosine_similarity(vecs)


In [11]:
merge_map = {}

for i, id_i in enumerate(ids):
    for j, id_j in enumerate(ids):
        if i < j and sim[i, j] >= 0.80:
            merge_map[id_j] = id_i

merge_bc = spark.sparkContext.broadcast(merge_map)

@F.udf("int")
def merge_udf(c):
    return merge_bc.value.get(c, c)

df_final = df_clean.withColumn("macro_final", merge_udf("macro_clean"))


In [12]:
df_final.select(
    "url",
    "macro_cluster",
    "macro_clean",
    "macro_final",
    "cosine_sim",
    "sim_cutoff"
).show(50, truncate=False)


+-----------------------------------------------------------------------------------------------------------------------------------------------+-------------+-----------+-----------+-------------------+-------------------+
|url                                                                                                                                            |macro_cluster|macro_clean|macro_final|cosine_sim         |sim_cutoff         |
+-----------------------------------------------------------------------------------------------------------------------------------------------+-------------+-----------+-----------+-------------------+-------------------+
|http://www.reuters.com/article/lloyds-ceo-expenses-idUSL3N1AR3W1                                                                               |5            |5          |5          |0.29684327969788954|0.21893684723611265|
|http://www.reuters.com/article/us-commodities-copper-trade-idUSKBN1770VU                               

In [13]:
#sanity check
df_final.groupBy("macro_final").count().orderBy("macro_final").show(200, truncate=False)
df_final.select("macro_final").distinct().count()

before = df_final.count()  # or df_join.count()
after = df_final.filter("macro_final != -1").count()
print("Kept:", after, "Dropped:", before - after, "Percent kept:", after / before)


+-----------+------+
|macro_final|count |
+-----------+------+
|-1         |389729|
|0          |214195|
|1          |67298 |
|2          |122231|
|3          |181879|
|4          |158493|
|5          |173369|
|6          |200545|
|7          |136944|
|8          |188747|
|9          |115651|
+-----------+------+



Kept: 1559352 Dropped: 389729 Percent kept: 0.8000447390334214


In [14]:
#saving cleaned clusters
df_macro_clean = df_final.select("url", "macro_final")

CLUSTER_DIR = ROOT / "clustering"
CLUSTER_MACRO_CLEAN = CLUSTER_DIR / "cluster_macro_cleaned"

print("Saving cleaned macro clusters to:", CLUSTER_MACRO_CLEAN)


(
    df_macro_clean
        .repartition(16)        
        .write
        .format("delta")
        .mode("overwrite")
        .save(str(CLUSTER_MACRO_CLEAN))
)

print("Cleaned macro clusters saved.")


Saving cleaned macro clusters to: /home/david/School/CapStone/clustering/cluster_macro_cleaned


Cleaned macro clusters saved.


# micro clusters

In [1]:
# ============================================================
# SAFE LOADING + FEATURE ASSEMBLY + SCALING PIPELINE
# ============================================================

from pathlib import Path
from pyspark.sql import SparkSession, functions as F
from delta import configure_spark_with_delta_pip
from pyspark.ml.feature import VectorAssembler, StandardScaler


# ------------------------------------------------------------
# Paths
# ------------------------------------------------------------
NB_DIR = Path.cwd()
ROOT = NB_DIR.parent

GOLD_DELTA   = ROOT / "pipelines" / "gold" / "gold_delta"
MACRO_DELTA  = ROOT / "clustering" / "cluster_macro_cleaned"

print("Gold path:", GOLD_DELTA)
print("Macro cluster path:", MACRO_DELTA)


# ------------------------------------------------------------
# Spark Session
# ------------------------------------------------------------
builder = (
    SparkSession.builder
        .appName("stage1_features")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .config("spark.driver.memory", "8g")
        .config("spark.sql.shuffle.partitions", "16")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()
print("Spark initialized.")


# ------------------------------------------------------------
# Load Delta Tables
# ------------------------------------------------------------
df_gold  = spark.read.format("delta").load(str(GOLD_DELTA))
df_macro = spark.read.format("delta").load(str(MACRO_DELTA))

# Remove legacy macro_cluster if present
if "macro_cluster" in df_gold.columns:
    df_gold = df_gold.drop("macro_cluster")
    print("Removed legacy macro_cluster from GOLD.")

print("Gold rows:", df_gold.count())
print("Macro rows:", df_macro.count())


# ------------------------------------------------------------
# Join once (safe)
# ------------------------------------------------------------
df_join = df_gold.join(df_macro, on="url", how="inner")
print("Joined total rows:", df_join.count())


# ------------------------------------------------------------
# Stage-1 Features
# ------------------------------------------------------------
stage1_pca   = [f"pca_{i}" for i in range(50)]
stage1_ner   = ["ner_org", "ner_gpe", "ner_person", "ner_money"]
stage1_lm    = ["lm_pos", "lm_neg", "lm_unc", "lm_lit", "lm_con"]
stage1_basic = ["len_chars_gold", "num_words", "avg_word_len"]

stage1_cols = stage1_pca + stage1_ner + stage1_lm + stage1_basic

print("Stage-1 feature count:", len(stage1_cols))


# ------------------------------------------------------------
# Select minimal columns needed before vector assembly
# ------------------------------------------------------------
df1 = df_join.select("url", "macro_final", *stage1_cols)

print("df1 count (safe):", df1.count())
df1.show(5, truncate=False)


# ------------------------------------------------------------
# VectorAssembler → features_raw (unscaled)
# ------------------------------------------------------------
assembler = VectorAssembler(
    inputCols=stage1_cols,
    outputCol="features_raw"
)

df2 = assembler.transform(df1)
print("VectorAssembler: features_raw created.")


# ------------------------------------------------------------
# StandardScaler → features (scaled vector)
# Fit ONLY on a sample (massive RAM saver)
# ------------------------------------------------------------
TOTAL_ROWS   = df2.count()
SAMPLE_SIZE  = 200_000

sample_frac  = min(1.0, SAMPLE_SIZE / TOTAL_ROWS)
df_sample    = df2.sample(False, sample_frac)

print(f"Scaler sample fraction: {sample_frac:.4f}")
print("Sample row count:", df_sample.count())

scaler = StandardScaler(
    inputCol="features_raw",
    outputCol="features",
    withMean=True,
    withStd=True
)

scaler_model = scaler.fit(df_sample)
df3 = scaler_model.transform(df2)

print("Scaling complete — 'features' column ready.")


# ------------------------------------------------------------
# Final Stage-1 Feature Table
# ------------------------------------------------------------
df_final = df3.select(
    "url",
    "macro_final",
    "features"        # 67-dim scaled vector
)

print("Final feature table ready:")
df_final.show(5, truncate=False)
print("Rows:", df_final.count())


# df_final is now safe to use in the microcluster loop.


Gold path: /home/david/School/CapStone/pipelines/gold/gold_delta
Macro cluster path: /home/david/School/CapStone/clustering/cluster_macro_cleaned


25/12/07 13:41:22 WARN Utils: Your hostname, david-ThinkPad-T490 resolves to a loopback address: 127.0.1.1; using 172.16.0.186 instead (on interface wlp0s20f3)
25/12/07 13:41:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/david/School/CapStone/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/david/.ivy2/cache
The jars for the packages stored in: /home/david/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2e9a78eb-c1cf-4737-b969-da87a02c625a;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 673ms :: artifacts dl 70ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0  

Spark initialized.


...

Scaler sample fraction: 0.1026


Sample row count: 199636


Scaling complete — 'features' column ready.
Final feature table ready:


+------------------------------------------------------------------------------------------------------------+-----------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Rows: 1949081


In [2]:
from pyspark.sql import functions as F
import numpy as np
from sklearn.cluster import MiniBatchKMeans

MICRO_OUT = ROOT / "clustering" / "microclusters_kmeans"
spark.sql("SET spark.sql.shuffle.partitions=16")


# ------------------------------------------------------------
# Initialise empty Delta table (safe)
# ------------------------------------------------------------
(
    spark.createDataFrame([], "url STRING, macro_final INT, microcluster INT")
         .write
         .format("delta")
         .mode("overwrite")
         .save(str(MICRO_OUT))
)


# ------------------------------------------------------------
# Load the final feature table once
# ------------------------------------------------------------
df = df_final       # url, macro_final, features (scaled)
macro_ids = [r[0] for r in df.select("macro_final").distinct().collect()]


# ------------------------------------------------------------
# Microcluster hyperparameters (REDUCED)
# ------------------------------------------------------------
MICRO_K    = 5      # <<--- only ~5-7 micro-themes per macro cluster
BATCH_SIZE = 2048
MAX_ITER   = 100


# ------------------------------------------------------------
# Main loop
# ------------------------------------------------------------
for cid in macro_ids:
    print(f"\n=== Processing macro cluster {cid} ===")

    # Filter by macro cluster
    df_c = (
        df
          .filter(F.col("macro_final") == cid)
          .select("url", "features")
    )

    count_c = df_c.count()
    print("Rows:", count_c)
    if count_c == 0:
        continue

    # --- FIXED SMALL K ---
    micro_k = MICRO_K
    print(f"Using fixed K={micro_k} microclusters.")

    # Convert Spark → Pandas → numpy matrix
    pdf = df_c.toPandas()
    X = np.vstack([v.toArray() for v in pdf["features"]])

    # Fast clustering
    model = MiniBatchKMeans(
        n_clusters=micro_k,
        batch_size=BATCH_SIZE,
        n_init="auto",
        max_iter=MAX_ITER
    )

    labels = model.fit_predict(X)

    # Skinny output
    out_pdf = pdf[["url"]].copy()
    out_pdf["macro_final"]  = cid
    out_pdf["microcluster"] = labels.astype(int)

    out_spark = spark.createDataFrame(out_pdf)

    # Ensure schema matches Delta table
    out_spark = (
        out_spark
            .withColumn("macro_final", F.col("macro_final").cast("int"))
            .withColumn("microcluster", F.col("microcluster").cast("int"))
    )

    (
        out_spark
            .write
            .format("delta")
            .mode("append")
            .save(str(MICRO_OUT))
    )

    print(f"Saved {len(labels)} rows to microcluster table.")

    # Hard memory drop
    del pdf, X, labels, model, out_pdf, out_spark
    import gc; gc.collect()

print("\nALL MACRO CLUSTERS DONE (MiniBatchKMeans).")



=== Processing macro cluster 5 ===


Rows: 173369
Using fixed K=5 microclusters.


25/12/07 13:46:23 WARN TaskSetManager: Stage 154 contains a task of very large size (2343 KiB). The maximum recommended task size is 1000 KiB.


Saved 173369 rows to microcluster table.

=== Processing macro cluster 3 ===


Rows: 181879
Using fixed K=5 microclusters.


...

25/12/07 13:56:49 WARN TaskSetManager: Stage 424 contains a task of very large size (2346 KiB). The maximum recommended task size is 1000 KiB.


Saved 188747 rows to microcluster table.

=== Processing macro cluster 0 ===


Rows: 214195
Using fixed K=5 microclusters.


25/12/07 13:58:04 WARN TaskSetManager: Stage 461 contains a task of very large size (2863 KiB). The maximum recommended task size is 1000 KiB.


Saved 214195 rows to microcluster table.

ALL MACRO CLUSTERS DONE (MiniBatchKMeans).


### Final Merging
- see MicroClusters EDA for reasoning here

In [ ]:
THEME_MAP = {
    # 1. Corporate Earnings & Financial Performance
    (0, 3): "Corporate Earnings & Financial Performance",
    (0, 4): "Corporate Earnings & Financial Performance",
    (3, 2): "Corporate Earnings & Financial Performance",

    # 2. Financial Markets & Trading
    (8, 2): "Financial Markets & Trading",
    (9, 1): "Financial Markets & Trading",
    (9, 3): "Financial Markets & Trading",
    (9, 4): "Financial Markets & Trading",

    # 3. Macroeconomic Trends & Outlook
    (8, 3): "Macroeconomic Trends & Outlook",
    (3, 3): "Macroeconomic Trends & Outlook",
    (4, 3): "Macroeconomic Trends & Outlook",

    # 4. Energy & Commodities Markets
    (0, 1): "Energy & Commodities Markets",
    (9, 2): "Energy & Commodities Markets",

    # 5. Corporate M&A & Private Equity
    (5, 1): "Corporate M&A & Private Equity",
    (5, 2): "Corporate M&A & Private Equity",

    # 6. Biotech & Healthcare Business
    (3, 0): "Biotech & Healthcare Business",

    # 7. Tech Industry & Innovation
    (0, 2): "Tech Industry & Innovation",
    (5, 0): "Tech Industry & Innovation",
    (5, 3): "Tech Industry & Innovation",

    # 8. Retail, Real Estate & Consumer
    (3, 1): "Retail, Real Estate & Consumer",
    (3, 4): "Retail, Real Estate & Consumer",
    (5, 4): "Retail, Real Estate & Consumer",

    # 9. US Domestic Politics
    (1, 1): "US Domestic Politics",
    (1, 3): "US Domestic Politics",
    (1, 4): "US Domestic Politics",
    (6, 4): "US Domestic Politics",

    # 10. Political Scandals & Legal Issues
    (1, 0): "Political Scandals & Legal Issues",
    (6, 1): "Political Scandals & Legal Issues",
    (6, 2): "Political Scandals & Legal Issues",
    (6, 3): "Political Scandals & Legal Issues",
    (7, 0): "Political Scandals & Legal Issues",
    (7, 1): "Political Scandals & Legal Issues",
    (7, 4): "Political Scandals & Legal Issues",

    # 11. International Trade & Relations
    (2, 0): "International Trade & Relations",
    (2, 3): "International Trade & Relations",
    (2, 4): "International Trade & Relations",

    # 12. Global Geopolitics & Security
    (2, 1): "Global Geopolitics & Security",
    (2, 2): "Global Geopolitics & Security",
    (4, 4): "Global Geopolitics & Security",

    # 13. UK & Brexit
    (4, 0): "UK & Brexit",
    (9, 0): "UK & Brexit",

    # 14. Health Policy & Public Health Crises
    (1, 2): "Health Policy & Public Health Crises",
    (4, 2): "Health Policy & Public Health Crises",

    # 15. Civil Rights & Justice Reform
    (7, 2): "Civil Rights & Justice Reform",
    (7, 3): "Civil Rights & Justice Reform",

    # 16. General / Mixed News
    (0, 0): "General / Mixed News",
    (4, 1): "General / Mixed News",
    (6, 0): "General / Mixed News",
    (8, 0): "General / Mixed News",
    (8, 1): "General / Mixed News",
    (8, 4): "General / Mixed News",
}


In [ ]:
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from delta import configure_spark_with_delta_pip


# Paths
NB_DIR = Path.cwd()
ROOT = NB_DIR.parent

CLUSTER_MICRO = ROOT / "clustering" / "microclusters_kmeans"

print("Notebook directory:", NB_DIR)
print("Project root:", ROOT)
print("Micro Cluster Delta path:", CLUSTER_MICRO)


# Spark session
builder = (
    SparkSession.builder
        .appName("assign_thematic_clusters")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
        .config("spark.driver.memory", "8g")
        .config("spark.sql.shuffle.partitions", "16")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()
print("Spark session initialised.")


# Load microclusters delta
df_micro = spark.read.format("delta").load(str(CLUSTER_MICRO))
print("Loaded microcluster table:")
df_micro.show(5, truncate=False)


# Final numeric mapping: (macro, micro) -> theme_id (1–16)
THEME_MAP = {
    (0, 3): 1, (0, 4): 1, (3, 2): 1,

    (8, 2): 2, (9, 1): 2, (9, 3): 2, (9, 4): 2,

    (8, 3): 3, (3, 3): 3, (4, 3): 3,

    (0, 1): 4, (9, 2): 4,

    (5, 1): 5, (5, 2): 5,

    (3, 0): 6,

    (0, 2): 7, (5, 0): 7, (5, 3): 7,

    (3, 1): 8, (3, 4): 8, (5, 4): 8,

    (1, 1): 9, (1, 3): 9, (1, 4): 9, (6, 4): 9,

    (1, 0): 10, (6, 1): 10, (6, 2): 10, (6, 3): 10,
    (7, 0): 10, (7, 1): 10, (7, 4): 10,

    (2, 0): 11, (2, 3): 11, (2, 4): 11,

    (2, 1): 12, (2, 2): 12, (4, 4): 12,

    (4, 0): 13, (9, 0): 13,

    (1, 2): 14, (4, 2): 14,

    (7, 2): 15, (7, 3): 15,

    (0, 0): 16, (4, 1): 16, (6, 0): 16,
    (8, 0): 16, (8, 1): 16, (8, 4): 16,
}


# UDF returning integer theme ID
def map_theme(mac, mic):
    return THEME_MAP.get((mac, mic), -1)   # -1 marks unknown combos

theme_udf = udf(map_theme, IntegerType())


# Assign numeric thematic cluster id
df_themes = df_micro.withColumn(
    "thematic_cluster",
    theme_udf("macro_final", "microcluster")
)

print("Assigned numeric thematic clusters (1–16):")
df_themes.show(10, truncate=False)


Notebook directory: /home/david/School/CapStone/clustering
Project root: /home/david/School/CapStone
Micro Cluster Delta path: /home/david/School/CapStone/clustering/microclusters_kmeans


25/12/07 21:17:22 WARN Utils: Your hostname, david-ThinkPad-T490 resolves to a loopback address: 127.0.1.1; using 172.16.0.186 instead (on interface wlp0s20f3)
25/12/07 21:17:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/david/School/CapStone/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/david/.ivy2/cache
The jars for the packages stored in: /home/david/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ebc39056-ad16-44f7-800c-843f61149f0c;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 497ms :: artifacts dl 26ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0  

Spark session initialised.


25/12/07 21:17:41 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


Loaded microcluster table:


25/12/07 21:17:49 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------------------------------------------------------------------------------+-----------+------------+
|url                                                                              |macro_final|microcluster|
+---------------------------------------------------------------------------------+-----------+------------+
|https://www.vox.com/2016/9/2/12769958/colin-kaepernick-eric-reid-protest         |6          |0           |
|https://www.vox.com/2016/9/23/13030628/hillary-clinton-ads-trump                 |6          |3           |
|https://www.vox.com/2016/9/4/12772748/trump-latino-voters-immigration            |6          |3           |
|https://www.vox.com/2016/9/7/12842998/trump-iraq-libya-lie-command-in-chief-forum|6          |3           |
|https://www.vox.com/2017/1/24/14371162/vox-sentences-trump-supreme-court         |6          |3           |
+---------------------------------------------------------------------------------+-----------+------------+
only showing top 5 

+----------------------------------------------------------------------------------------------------------------------------------+-----------+------------+----------------+
|url                                                                                                                               |macro_final|microcluster|thematic_cluster|
+----------------------------------------------------------------------------------------------------------------------------------+-----------+------------+----------------+
|https://www.vox.com/2016/9/2/12769958/colin-kaepernick-eric-reid-protest                                                          |6          |0           |16              |
|https://www.vox.com/2016/9/23/13030628/hillary-clinton-ads-trump                                                                  |6          |3           |10              |
|https://www.vox.com/2016/9/4/12772748/trump-latino-voters-immigration                                                       

In [2]:
# ============================
# Save final thematic clusters
# ============================

CLUSTER_THEMATIC = ROOT / "clustering" / "thematic_clusters"

print("Saving thematic clusters to:", CLUSTER_THEMATIC)

(
    df_themes
        .select("url", "macro_final", "microcluster", "thematic_cluster")
        .repartition(16)
        .write
        .format("delta")
        .mode("overwrite")
        .save(str(CLUSTER_THEMATIC))
)

print("Thematic clusters saved successfully.")


Saving thematic clusters to: /home/david/School/CapStone/clustering/thematic_clusters


Thematic clusters saved successfully.


25/12/08 09:48:25 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 38703818 ms exceeds timeout 120000 ms
25/12/08 09:48:25 WARN SparkContext: Killing executors is not supported by current scheduler.
25/12/08 09:48:25 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint